### Using
##### https://github.com/Rootex/99-prolog-problems
##### DELETED BECAUSE IT WAS .MD NOT CODE https://github.com/Duke-PL-Course/Prolog/blob/master/2013-02-26-prolog-review-problem.md
##### https://github.com/triska/the-power-of-prolog/
##### https://github.com/Anniepoo/prolog-examples
##### https://github.com/dragonwasrobot/learn-prolog-now-exercises
##### https://github.com/xmonader/prolog-rands

##### DO ALSO https://github.com/dtonhofer/prolog_notes/tree/master/prolog_exercises (NOT DOWNLOADED YET)

In [1]:
import json
import torch
import torch.nn as nn
import random
import os
from tqdm import tqdm
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [3]:
dev_dict = json.load(open('../data/coqa-dev-v1.0.json', encoding='utf8'))

In [4]:
files = os.listdir('../data/prolog_examples')

In [5]:
text = ''
for file in files:
    text += open(os.path.join('../data/prolog_examples/', file)).read()

In [6]:
import re

p = re.compile('%(.*)\n')
text = p.sub('/* \g<1> */\n', text)

In [7]:
tokens = tokenizer.encode(text, return_tensors='pt')


Token indices sequence length is longer than the specified maximum sequence length for this model (190891 > 1024). Running this sequence through the model will result in indexing errors


In [8]:
train_list = []
for index in range(0, len(tokens[0]), 1024):
    train_list.append(tokens[0][index: index + 1024])

In [9]:
random.shuffle(train_list)

In [10]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def batchify(data, n):
    len_dict = {}
    for item in data:
        length = len(item)
        try:
            len_dict[length].append(item)
        except:
            len_dict[length] = [item]

    batch_chunks = []
    for k in len_dict.keys():
        vectors = len_dict[k]
        batch_chunks += chunks(vectors, n)

    batches = []
    for chunk in batch_chunks:
        inputs = torch.stack([item for item in chunk])
        batches.append((inputs))

    return batches

In [11]:
train_batches = batchify(train_list, 1)

In [12]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=4e-5)

In [13]:
def train(train_model, batches, optimizer, criterion):
    total_loss = 0.
    for i, batch in tqdm(enumerate(batches), total=len(batches)):
        model.train()
        inputs = batch
        optimizer.zero_grad()
        loss = train_model(inputs.cuda(), labels=inputs.cuda())[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(train_model.parameters(), 0.5)
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(batches)

def test(test_model, batches):
    total_loss = 0.
    for i, batch in tqdm(enumerate(batches), total=len(batches)):
        test_model.eval()
        inputs = batch
        loss = test_model(inputs, labels=inputs)[0]
        total_loss += loss.item()

    return total_loss / len(batches)

In [14]:
from torch.optim.lr_scheduler import StepLR

random.shuffle(train_batches)
scheduler = StepLR(optimizer, step_size=10, gamma=0.8)
for epoch in range(100):
    random.shuffle(train_batches)
    loss = train(model, train_batches, optimizer, criterion)
    print('Epoch:', epoch, 'Loss:', loss)
    torch.save({'epoch': epoch,
                'model_state_dict': model.state_dict()},
                'save_prolog' + str(epoch))
    scheduler.step()

100%|██████████| 187/187 [00:32<00:00,  5.71it/s]


Epoch: 0 Loss: 1.9179175253857903


100%|██████████| 187/187 [00:33<00:00,  5.63it/s]


Epoch: 1 Loss: 1.5857053083531998


100%|██████████| 187/187 [00:33<00:00,  5.61it/s]


...

Epoch: 97 Loss: 0.07286292087177143


100%|██████████| 187/187 [00:35<00:00,  5.31it/s]


Epoch: 98 Loss: 0.07213542868886243


100%|██████████| 187/187 [00:35<00:00,  5.31it/s]


Epoch: 99 Loss: 0.07096756021566888


In [21]:
def generate_code(model, prompt):
    model.eval()
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 1024
    tokens_length = tokens.shape[1]
    output = model.generate(
             tokens.cuda(),
             max_length=min(1024, tokens_length + _length),
             temperature=0.5,
             #num_beams=3,
             pad_token_id=50256
    )
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    offset = len(prompt)
    return output

In [37]:
checkpoint = torch.load('save_prolog' + str(40))
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [38]:
prompt = "/* Model the following: Sonny is a white cat that sleeps in a barn */\n\n"
print(generate_code(model, prompt))

/* Model the following: Sonny is a white cat that sleeps in a barn */

     ?- cat(X).
      /* @ X = cat. */

/* % Who says that bob likes to be cuddled by the sly? */
/* % He's a real a__hole. */
/* % But I saw a fellow named Carl standing in the park next to the river */


/* % Who says that Mr. Boddy likes to be cuddled by the sly? */
/* % Carl is real a__hole. */
/* % WTF? */
/* % You're a real a__hole. */
/* % You've heard of the famous detective story */
/* % that Mr. Boddy was the victim's friend. */
/* % Who says that fellow John is a real a__hole? */
/* % John is a real a__hole. */

/* % WTF? */
/* % You've read the detective story, and you don't know who did it. */
/* % Who says that fellow John is a real a__hole? */

/* % You've read the Burt story, and you don't know who did it either. */

/* % WTF? */
/* % You've read the WTF? rule, and you don't know who did it either. */

/* % % You've read the rules, and you don't know who did it either. */

/* % WTF? */
/* % You've re